# Tutorial para usar QIIME en PATUNG

Tutorial para correr QIIME en el servidor de PATUNG del LANCIS.
Se asume que el usuario ya tiene cuenta y contraseña del servidor de PATUNG.
El usuario debe tener conocimiento de BASH para crear y mover archivos (no es cierto, pongo los comandos aquí pero si ayuda).

* Nota: si no te gusta Linux (como a mi) todo el proceso se puede realizar en Windows 10 usando el subsistema Linux/Ubuntu sin usar máquina virtual. 


Para empezar este tutorial debemos estar dentro de nuestra sesión de PATUNG y corriendo el Notebook remotamente en nuestra máquina local. Para esto por favor sigue las instrucciones en el siguiente link: __```url```__



## Usando QIIME

Una ve

Import seqs

In [ ]:
{
echo '#!/bin/bash'
echo 'source /srv/home/anavarro/miniconda3/bin/activate qiime2'

echo 'qiime tools import \'
echo "--type 'SampleData[PairedEndSequencesWithQuality]' \\"
echo '--input-path $1 \'
echo '--source-format PairedEndFastqManifestPhred33 \'
echo '--output-path $2 \'

} > 1_import_seqs.sh

In [ ]:
{
echo 'executable 	= /srv/home/anavarro/metag_granulos_hidro/3_ejecutables/1_import_seqs.sh'
echo 'getenv		= True'
echo 'Arguments 	= /srv/home/anavarro/metag_granulos_hidro/1_secuencias/seqs_manifest.txt /srv/home/anavarro/metag_granulos_hidro/2_resultados/1_demultiplexed_pairedEnd_seqs.qza'
echo 'output		= /srv/home/anavarro/metag_granulos_hidro/4_outs/1_import_seqs$(Process).out '
echo 'log  		= /srv/home/anavarro/metag_granulos_hidro/5_logs/1_import_seqs$(Process).log'
echo 'error		= /srv/home/anavarro/metag_granulos_hidro/6_errores/1_import_seqs$(Process).error'

echo 'request_cpus	=5'

echo 'queue'
} > 1_import_seqs_condor.condor

In [ ]:
condor_submit 1_import_seqs_condor.condor

In [ ]:
condor_q

Join pair ends

In [ ]:
{
echo '#!/bin/bash'
echo 'source /srv/home/anavarro/miniconda3/bin/activate qiime2'

echo 'qiime vsearch join-pairs \'
echo '--i-demultiplexed-seqs $1 \'
echo '--o-joined-sequences $2 \'
echo '--p-allowmergestagger \'
echo '--verbose'
} > 2_vsearch_join-pairs.sh

In [ ]:
{
echo 'executable 	= /srv/home/anavarro/metag_granulos_hidro/3_ejecutables/2_vsearch_join-pairs.sh'
echo 'getenv		= True'
echo 'Arguments 	= /srv/home/anavarro/metag_granulos_hidro/2_resultados/1_demultiplexed_pairedEnd_seqs.qza /srv/home/anavarro/metag_granulos_hidro/2_resultados/2_demultiplexed_joined_seqs_wq.qza'
echo 'output		= /srv/home/anavarro/metag_granulos_hidro/4_outs/2_vsearch_join$(Process).out '
echo 'log  		= /srv/home/anavarro/metag_granulos_hidro/5_logs/2_vsearch_join$(Process).log'
echo 'error		= /srv/home/anavarro/metag_granulos_hidro/6_errores/2_vsearch_join$(Process).error'

echo 'request_cpus	=5'

echo 'queue'
} > 2_vsearch_join-pairs_submit.condor

In [ ]:
condor_submit 2_vsearch_join-pairs_submit.condor

Quality filter

In [ ]:
{
echo '#!/bin/bash'
echo 'source /srv/home/anavarro/miniconda3/bin/activate qiime2'

echo 'qiime quality-filter q-score-joined \'
echo '--i-demux /srv/home/anavarro/metag_granulos_hidro/2_resultados/2_demultiplexed_joined_seqs_wq.qza \'
echo '--p-min-quality 19 \'
echo '--p-quality-window 5 \'
echo '--p-min-length-fraction 0.5 \'
echo '--p-max-ambiguous 10 \'
echo '--output-dir /srv/home/anavarro/metag_granulos_hidro/2_resultados/3_q_filter_output \'
echo '--verbose'
} > 3_quality-filter.sh

In [ ]:
{
echo 'executable 	= /srv/home/anavarro/metag_granulos_hidro/3_ejecutables/3_quality-filter.sh'
echo 'getenv		= True'
echo 'output		= /srv/home/anavarro/metag_granulos_hidro/4_outs/3_quality-filter$(Process).out'
echo 'log  		= /srv/home/anavarro/metag_granulos_hidro/5_logs/3_quality-filter$(Process).log'
echo 'error		= /srv/home/anavarro/metag_granulos_hidro/6_errores/3_quality-filter$(Process).error'

echo 'request_cpus	=5'

echo 'queue'
} > 3_quality-filter_submbit.condor

In [ ]:
condor_submit 3_quality-filter_submbit.condor

Deblur denoise

In [ ]:
{
echo '#!/bin/bash'
echo 'source /srv/home/anavarro/miniconda3/bin/activate qiime2'

echo 'qiime deblur denoise-16S \'
echo '--i-demultiplexed-seqs /srv/home/anavarro/metag_granulos_hidro/2_resultados/3_q_filter_output/filtered_sequences.qza \'
echo '--p-trim-length 250 \'
echo '--p-sample-stats \'
echo '--p-jobs-to-start 40 \'
echo '--output-dir /srv/home/anavarro/metag_granulos_hidro/2_resultados/4_deblur_output \'
echo '--verbose'
} > 4_deblur-denoise.sh

In [ ]:
{
echo 'executable 	= /srv/home/anavarro/metag_granulos_hidro/3_ejecutables/4_deblur-denoise.sh'

echo 'getenv		= True'
echo 'output		= /srv/home/anavarro/metag_granulos_hidro/4_outs/4_deblur-denoise$(Process).out'
echo 'log  		= /srv/home/anavarro/metag_granulos_hidro/5_logs/4_deblur-denoise$(Process).log'
echo 'error		= /srv/home/anavarro/metag_granulos_hidro/6_errores/4_deblur-denoise$(Process).error'

echo 'request_cpus	=5'

echo 'queue'
} > 4_deblur-denoise_submbit.condor

In [ ]:
condor_submit 4_deblur-denoise_submbit.condor

cluster-features-de-novo

In [ ]:
{
echo '#!/bin/bash'
echo 'source /srv/home/anavarro/miniconda3/bin/activate /srv/home/anavarro/miniconda3/envs/qiime2'

echo 'qiime vsearch cluster-features-de-novo \'
echo '--i-sequences /srv/home/anavarro/metag_granulos_hidro/2_resultados/4_deblur_output/representative_sequences.qza \'
echo '--i-table /srv/home/anavarro/metag_granulos_hidro/2_resultados/4_deblur_output/table.qza \'
echo '--p-perc-identity 0.97 \'
echo '--p-threads 0 \'
echo '--output-dir /srv/home/anavarro/metag_granulos_hidro/2_resultados/5_vsearch_output \'
echo '--verbose'
} > 5_vsearch_cluster_denovo.sh

In [ ]:
{
echo 'executable 	= /srv/home/anavarro/metag_granulos_hidro/3_ejecutables/5_vsearch_cluster_denovo.sh'
echo 'getenv		= True'

echo 'output		= /srv/home/anavarro/metag_granulos_hidro/4_outs/5_vsearch_denovo$(Process).out '
echo 'log  		= /srv/home/anavarro/metag_granulos_hidro/5_logs/5_vsearch_denovo$(Process).log'
echo 'error		= /srv/home/anavarro/metag_granulos_hidro/6_errores/5_vsearch_denovo$(Process).error'

echo 'request_cpus	=5'

echo 'queue'

} > 5_vsearch_cluster_denovo_submbit.condor

In [ ]:
condor_submit 5_vsearch_cluster_denovo_submbit.condor

In [ ]:
{
echo '#!/bin/bash'

echo 'qiime alignment mafft \'
echo '--i-sequences /srv/home/anavarro/metag_granulos_hidro/2_resultados/5_vsearch_output/clustered_sequences.qza \'
echo '--p-n-threads -1 \'
echo '--output-dir /srv/home/anavarro/metag_granulos_hidro/2_resultados/6_alignment_output \'
echo '--verbose'

} > 6_alignment_mafft.sh

In [ ]:
{
echo 'executable 	= /srv/home/anavarro/metag_granulos_hidro/3_ejecutables/6_alignment_mafft.sh'
echo 'getenv		= True'

echo 'output		= /srv/home/anavarro/metag_granulos_hidro/4_outs/6_alignment_mafft$(Process).out '
echo 'log  		= /srv/home/anavarro/metag_granulos_hidro/5_logs/6_alignment_mafft$(Process).log'
echo 'error		= /srv/home/anavarro/metag_granulos_hidro/6_errores/6_alignment_mafft$(Process).error'

echo 'request_cpus	= 5'

echo 'queue'

} > 6_alignment_mafft_submbit.condor

In [ ]:
condor_submit 6_alignment_mafft_submbit.condor

qiime feature-classifier classify-sklearn

In [ ]:
Bajar gg-13-8-99-nb-classifier.qza

In [ ]:
{
echo '#!/bin/bash'

echo 'qiime feature-classifier classify-sklearn \'
echo '--i-reads /srv/home/anavarro/metag_granulos_hidro/2_resultados/5_vsearch_output/clustered_sequences.qza \'
echo '--i-classifier /srv/home/anavarro/metag_granulos_hidro/gg-13-8-99-nb-classifier.qza \'
echo '--p-n-jobs -1 \'
echo '--p-confidence 0.8 \'
echo '--output-dir /srv/home/anavarro/metag_granulos_hidro/2_resultados/7_taxonomy_output \'
echo '--verbose'

} > 7_taxonomy.sh

In [ ]:
{
echo 'executable 	= /srv/home/anavarro/metag_granulos_hidro/3_ejecutables/7_taxonomy.sh'
echo 'getenv		= True'
echo 'output		= /srv/home/anavarro/metag_granulos_hidro/4_outs/7_taxonomy$(Process).out'
echo 'log  		= /srv/home/anavarro/metag_granulos_hidro/5_logs/7_taxonomy$(Process).log'
echo 'error		= /srv/home/anavarro/metag_granulos_hidro/6_errores/7_taxonomy$(Process).error'

echo 'request_cpus	= 5'
echo 'queue'
} > 7_taxonomy_submbit.condor

In [ ]:
Taxonomy silva

In [ ]:
{
echo '#!/bin/bash'

echo 'qiime feature-classifier classify-sklearn \'
echo '--i-reads /srv/home/anavarro/metag_granulos_hidro/2_resultados/5_vsearch_output/clustered_sequences.qza \'
echo '--i-classifier /srv/home/anavarro/metag_granulos_hidro/silva-119-99-nb-classifier.qza \'
echo '--p-n-jobs -1 \'
echo '--p-confidence 0.8 \'
echo '--output-dir /srv/home/anavarro/metag_granulos_hidro/2_resultados/7_taxonomy_output_silva \'
echo '--verbose'

} > 7_taxonomy_silva.sh

In [ ]:
{
echo 'executable 	= /srv/home/anavarro/metag_granulos_hidro/3_ejecutables/7_taxonomy_silva.sh'
echo 'getenv		= True'
echo 'output		= /srv/home/anavarro/metag_granulos_hidro/4_outs/7_taxonomy_silva$(Process).out'
echo 'log  		= /srv/home/anavarro/metag_granulos_hidro/5_logs/7_taxonomy_silva$(Process).log'
echo 'error		= /srv/home/anavarro/metag_granulos_hidro/6_errores/7_taxonomy_silva$(Process).error'

echo 'request_cpus	= 5'
echo 'queue'
} > 7_taxonomy_submbit_silva.condor

phylogeny fast tree

In [ ]:
{
echo '#!/bin/bash'

echo 'qiime phylogeny fasttree \'
echo '--i-alignment /srv/home/anavarro/metag_hidro2/2_resultados/6_alignment_output/alignment.qza \'
echo '--p-n-threads -1 \'
echo '--o-tree /srv/home/anavarro/metag_hidro2/2_resultados/10_tree_output \'
echo '--verbose'
} > 10_fasttree.sh

In [ ]:
{
echo 'executable 	= /srv/home/anavarro/metag_hidro2/3_ejecutables/10_fasttree.sh'
echo 'getenv		= True'
echo 'output		= /srv/home/anavarro/metag_hidro2/4_outs/10_fasttree$(Process).out'
echo 'log  		= /srv/home/anavarro/metag_hidro2/5_logs/10_fasttree$(Process).log'
echo 'error		= /srv/home/anavarro/metag_hidro2/6_errores/10_fasttree$(Process).error'


echo 'request_cpus	= 5'

echo 'queue'
} > 10_fasttree_submbit.condor

8 Exporting deblur/denoise output to biom file

In [ ]:
{
echo '#!/bin/bash'

echo 'qiime tools export \'
echo '/srv/home/anavarro/metag_hidro2/2_resultados/5_vsearch_output/clustered_table.qza \'
echo '--output-dir /srv/home/anavarro/metag_hidro2/2_resultados/8_biom/'
} > 8_export_biom.sh

In [ ]:
{
echo 'executable 	= /srv/home/anavarro/metag_hidro2/3_ejecutables/8_export_biom.sh'
echo 'getenv		= True'
echo 'output		= /srv/home/anavarro/metag_hidro2/4_outs/8_biom$(Process).out'
echo 'log  		= /srv/home/anavarro/metag_hidro2/5_logs/8_biom$(Process).log'
echo 'error		= /srv/home/anavarro/metag_hidro2/6_errores/8_biom$(Process).error'

echo 'request_cpus	= 5'

echo 'queue'
} > 8_export_biom.condor

10 Exporting feature table to TSV file

In [ ]:
{
echo '#!/bin/bash'

echo 'qiime tools export \'
echo '/srv/home/anavarro/metag_hidro2/2_resultados/7_taxonomy_vsearch_silva/classification.qza \'
echo '--output-dir /srv/home/anavarro/metag_hidro2/2_resultados/9_tsv_silva_vs'
} > 9_export_to_tsv_silva_vs.sh

In [ ]:
{
echo 'executable 	= /srv/home/anavarro/metag_granulos_hidro/3_ejecutables/10_export_to_tsv.sh'
echo 'getenv		= True'
echo 'output		= /srv/home/anavarro/metag_granulos_hidro/4_outs/10_export_tsv$(Process).out'
echo 'log  		= /srv/home/anavarro/metag_granulos_hidro/5_logs/10_export_tsv$(Process).log'
echo 'error		= /srv/home/anavarro/metag_granulos_hidro/6_errores/10_export_tsv$(Process).error'

echo 'request_cpus	= 5'

echo 'queue'
} > 10_export_to_tsv_condor.condor

11 Exporting qza tree to nwk

In [ ]:
{
echo '#!/bin/bash'

echo 'qiime tools export \'
echo '/srv/home/anavarro/metag_hidro2/2_resultados/10_tree_output.qza \'
echo '--output-dir /srv/home/anavarro/metag_hidro2/2_resultados/11_exported-tree'
} > 11_export_to_nwk.sh

In [ ]:
{
echo 'executable 	= /srv/home/anavarro/metag_hidro2/3_ejecutables/11_export_to_nwk.sh'
echo 'getenv		= True'
echo 'output		= /srv/home/anavarro/metag_hidro2/4_outs/11_export_to_nwk$(Process).out'
echo 'log  		= /srv/home/anavarro/metag_hidro2/5_logs/11_export_to_nwk$(Process).log'
echo 'error		= /srv/home/anavarro/metag_hidro2/6_errores/11_export_to_nwk$(Process).error'


echo 'request_cpus	= 5'

echo 'queue'
} > 11_export_to_nwk_condor.condor

In [ ]:
{
echo '#!/bin/bash'

echo 'biom add-metadata \'
echo '-i /srv/home/anavarro/metag_hidro2/2_resultados/8_biom/feature-table.biom  \'
echo '-o /srv/home/anavarro/metag_hidro2/2_resultados/12_1_table_w_tax_silva_vs.biom  \'
echo '--observation-metadata-fp /srv/home/anavarro/metag_hidro2/2_resultados/9_tsv_silva_vs/taxonomy.tsv \'
echo '--observation-header OTUID,taxonomy  \'
echo '--sc-separated taxonomy  \'
echo '--sample-metadata-fp /srv/home/anavarro/metag_hidro2/1_secuencias/metadata.txt  \'
echo '--sample-header SampleID,ciclo,sustrato,hidrogeno,acet '
} > 12_join_biom_silva_vs.sh

In [ ]:
{
echo '#!/bin/bash'
echo 'biom convert
echo '-i table.biom
echo '-o table.from_biom_w_taxonomy.txt
echo '--to-tsv
echo '--header-key taxonomy
} > 13_biom_to_otu_table_gg_sk.sh

In [ ]:
biom add-metadata -i /srv/home/anavarro/metag_hidro2/2_resultados/8_biom/feature-table.biom -o /srv/home/anavarro/metag_granulos_hidro/2_resultados/12_joined_biom/table_w_tax.biom --observation-metadata-fp /srv/home/anavarro/metag_granulos_hidro/2_resultados/10_exported-feature-table-tsv/taxonomy.tsv --observation-header OTUID,taxonomy --sc-separated taxonomy

In [ ]:
biom add-metadata -i /srv/home/anavarro/metag_granulos_hidro/2_resultados/12_biom_join/table.w_omd_smd.biom -o /srv/home/anavarro/metag_granulos_hidro/2_resultados/13_biom_join_smd/table_w_smd.biom --sample-metadata-fp /srv/home/anavarro/metag_granulos_hidro/1_secuencias/metadata.txt --sample-header SampleID,ciclo,sustrato,hidrogeno,acet